In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
username = 'vigrose'
password_edw = 'Slytherin~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [5]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [9]:
BOARD_QUERY = \
    f"""
    SELECT
    PARTY_ID_FROM,
    PARTY_ID_TO,
    BRD_ID,
    CERTIF_TYPE_CD,
    CERTIF_ID,
    REC_TYPE,
    ISS_DT,
    EXP_DT,
    REVERIFICATION_DT,
    OCCUR_TYPE_CD,
    OCCUR_TYPE_DESC,
    MOC_MET_RQT,
    MOC_PATHWAY_ID,
    ABMS_ID,
    CERT_ID
    FROM
    AMAEDW.BRD_CERTIF
    WHERE
    THRU_DT is null
    """
abms = pd.read_sql(con=AMAEDW, sql=BOARD_QUERY)

In [23]:
abms.BRD_ID.unique()

array([11, 15, 34, 16, 21, 17, 28, 29, 14, 31, 26, 20, 12, 24, 27, 32, 18,
       22, 23, 10, 13, 33, 25, 30], dtype=int64)

In [6]:
LIC_QUERY = \
    f"""
    SELECT
    PARTY_ID,
    STATE_ID,
    ISS_DT,
    EXP_DT,
    RNW_DT,
    DEGREE_CD,
    LIC_NBR,
    AS_OF_DT,
    FROM_DT,
    INSERT_TMSTP,
    UPDATE_TMSTP
    FROM
    AMAEDW.LIC
    WHERE
    THRU_DT is null
    AND
    DELETE_IND = 'N'
    """
license = pd.read_sql(con=AMAEDW, sql=LIC_QUERY)

In [7]:
license

,PARTY_ID,STATE_ID,ISS_DT,EXP_DT,RNW_DT,DEGREE_CD,LIC_NBR,AS_OF_DT,FROM_DT,INSERT_TMSTP,UPDATE_TMSTP
0,4501573,207,2013-06-19,2016-11-30,None,MD,256688,2017-04-28,2017-03-17 20:08:31,2017-03-17 20:08:31,2017-04-30 21:56:09.000000
1,3413481,200,2010-08-24,2016-12-31,None,DO,5101019029,2017-04-04,2017-03-17 20:08:31,2017-03-17 20:08:31,2017-04-12 02:06:55.000000
2,3586092,257,2005-06-04,2008-12-31,None,MD,MD426635,2018-10-19,2017-03-17 20:08:31,2017-03-17 20:08:31,2018-11-14 01:31:53.000000
3,3501324,247,1996-08-26,1997-01-15,None,None,106000035,2020-02-13,2017-03-17 20:08:31,2017-03-17 20:08:31,2020-02-15 04:03:51.000000
4,3326444,239,2014-09-12,2016-09-11,None,MD,BP20051463,2016-10-03,2016-10-11 11:34:12,2016-10-11 22:28:59,2021-11-15 13:51:09.425153
...,...,...,...,...,...,...,...,...,...,...,...
3705243,1799261,214,2011-04-19,2021-03-31,None,MD,12649,2021-04-08,2022-08-13 02:34:23,2022-08-13 02:34:23,2022-08-13 02:34:23.000000
3705244,2444373,231,2018-10-03,2022-06-30,None,MD,19237,2022-06-02,2022-08-13 02:34:23,2022-08-13 02:34:23,2022-08-13 02:34:23.000000
3705245,5137035,231,2019-12-04,2021-06-30,None,MD,20169,2021-05-05,2022-08-13 02:34:23,2022-08-13 02:34:23,2022-08-13 02:34:23.000000
3705246,15407577,225,2015-11-10,2024-06-30,None,DO,0102204347,2022-08-10,2022-08-13 02:34:23,2022-08-13 02:34:23,2022-08-13 02:34:23.000000


In [46]:
folders = pd.read_csv('../../Data/Measurement/License_Folder_Creation_July.csv')
state = pd.read_csv('../../Data/Measurement/STATE.csv')

In [12]:
IDS = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')
# ALL_ABMS = pd.merge(IDS, abms, left_on='PARTY_ID', right_on='PARTY_ID_FROM').drop_duplicates('CERTIF_ID')
ALL_LICENSE = pd.merge(state[['DESC', 'STATE_ID']], license, on='STATE_ID').drop_duplicates()
ALL_LICENSE = pd.merge(IDS, ALL_LICENSE, on='PARTY_ID').drop_duplicates()
# ALL_LICENSE = ALL_LICENSE.drop_duplicates(['LIC_NBR','DEGREE_CD','STATE_ID','PARTY_ID'])
# ALL_DEA = pd.merge(IDS, dea, on='PARTY_ID').drop_duplicates()
# ALL_NPI = pd.merge(IDS, npi, on='PARTY_ID').drop_duplicates()

In [47]:
license_degrees = []
lic_types = []
states = []

for x in folders.STATE:
    thing = x.split(' ')[-1]
    if thing in ['DO','MD']:
        license_degree = thing
        state = x.replace(f' {thing}','')
    else:
        license_degree = ''
        state = x
    if thing in ['TR','IR']:
        lic_type = 'Resident'
        state = x.replace(f' {thing}','')
    else:
        lic_type = ''
    license_degrees.append(license_degree)
    lic_types.append(lic_type)
    states.append(state)
folders['LICENSE_TYPE'] = lic_types
folders['LICENSE_DEGREE'] = license_degrees
folders['LIC_STATE'] = states
folders

,STATE,CREATION_DT,LICENSE_TYPE,LICENSE_DEGREE,LIC_STATE
0,Alabama,2022-07-20 07:08:17.114477,,,Alabama
1,Alaska,2022-07-01 09:21:24.046996,,,Alaska
2,Arizona DO,2022-07-11 09:40:51.622121,,DO,Arizona
3,Arizonia MD,2022-07-05 10:35:16.643136,,MD,Arizonia
4,Arkansas,2022-07-15 09:33:22.413930,,,Arkansas
5,California MD,2022-07-06 13:43:14.017229,,MD,California
6,Colorado,2022-07-07 15:54:55.743980,,,Colorado
7,Delaware,2022-07-05 05:48:15.334742,,,Delaware
8,Florida DO,2022-07-06 13:46:07.262469,,DO,Florida
9,Florida MD,2022-07-06 13:45:57.472455,,MD,Florida


In [94]:
lic_degrees = []
split_states = list(folders[folders.LICENSE_DEGREE.isin(['MD','DO'])].LIC_STATE)
for row in ALL_LICENSE.itertuples():
    degree = ''
    if row.DESC in (split_states):
        degree = row.DEGREE_CD
    lic_degrees.append(degree)
ALL_LICENSE['LICENSE_DEGREE'] = lic_degrees

In [93]:
license_types = []
training = list(folders[folders.LICENSE_TYPE=='Resident'].LIC_STATE)
print(training)
for row in ALL_LICENSE.itertuples():
    tp = ''
    if (row.DESC in training) & (row.type == 'Resident'):
        tp = row.type
    license_types.append(tp)
ALL_LICENSE['LICENSE_TYPE'] = license_types

['Kentucky', 'Mississippi', 'Oklahoma', 'Texas', 'Virginia']


In [95]:
ALL_LICENSE.LICENSE_TYPE.unique()

array(['', 'Resident'], dtype=object)

In [96]:
all_of_it = pd.merge(ALL_LICENSE, folders, left_on=['DESC', 'LICENSE_DEGREE','LICENSE_TYPE'], right_on=['LIC_STATE', 'LICENSE_DEGREE','LICENSE_TYPE'])

In [97]:
all_of_it.CREATION_DT = pd.to_datetime(all_of_it.CREATION_DT)

In [98]:
all_of_it['UPDATE_TIME'] = all_of_it.UPDATE_TMSTP - all_of_it.CREATION_DT 

In [100]:
all_of_it.ISS_DT = pd.to_datetime(all_of_it.ISS_DT, errors='coerce')

In [101]:
all_of_it['ISSUE_TO_INSERT'] = all_of_it.INSERT_TMSTP - all_of_it.ISS_DT

In [102]:
all_of_it['UPDATE_DAYS'] = [x.days for x in all_of_it['UPDATE_TIME']]

In [103]:
update_mean_by_state = all_of_it[all_of_it.UPDATE_DAYS>0].groupby('STATE').mean()[['UPDATE_DAYS']]

In [134]:
# all_of_it['INSERT_DAYS'] = [x.days for x in all_of_it['ISSUE_TO_INSERT']]
insert_mean_by_state = all_of_it[(all_of_it.INSERT_DAYS_x>0)&(all_of_it.ISS_DT>=datetime.date(2005,1,1))].groupby('STATE').mean()[['INSERT_DAYS_x']]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [135]:
all_of_it = pd.merge(all_of_it, insert_mean_by_state, on='STATE')
# all_of_it = pd.merge(all_of_it, update_mean_by_state, on='STATE')

In [128]:
datetime.date(2005,1,1)

datetime.date(2005, 1, 1)

In [149]:
(all_of_it[(all_of_it.INSERT_DAYS_x_x>0)&(all_of_it.ISS_DT>datetime.date(2006,1,1))].INSERT_DAYS_x_x.mean())
#mean = 2577.405897817647
#7.061386021418212

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


2435.8866645527078

In [117]:
all_of_it[all_of_it.UPDATE_DAYS_x>0].UPDATE_DAYS_x.mean()
#mean = 25.054526994910784

25.054526994910784

In [151]:
all_of_it.groupby('STATE')[['PARTY_ID']].count()

,PARTY_ID
STATE,
Alabama,41674
Alaska,13289
Arizona DO,3241
Arkansas,20984
California MD,231481
Colorado,55560
Delaware,18183
Florida DO,18158
Florida MD,152117


In [140]:
592727/2743122

0.21607752043110004

,UPDATE_DAYS
STATE,
Alabama,16.475000
Alaska,32.000000
Arizona DO,3.025499
Arkansas,4.010894
California MD,28.000471
Colorado,26.979238
Delaware,28.000974
Florida DO,35.001215
Florida MD,35.000158


In [141]:
all_of_it.to_csv('../../Data/Measurement/License_Timeliness_July.csv', index=False)

KeyboardInterrupt: 

In [ ]:
update_

In [17]:
DEA_QUERY = \
    f"""
    SELECT
    PARTY_ID,
    DEA_ACTIVITY_CD,
    DEA_SUB_ACTIVITY_CODE,
    SCHED_CD,
    EXP_DT,
    DEA_STATUS_ID,
    DEA_NBR
    FROM
    AMAEDW.DEA_REG
    WHERE
    THRU_DT is null
    AND
    DELETE_IND = 'N'
    """
dea = pd.read_sql(con=AMAEDW, sql=DEA_QUERY)

In [22]:
dea.PARTY_ID.unique()

array(['22N 33N 4 5', '2N  3N 4 5', '2   3   4', '33N 4 5', '22N 33N 4',
       '2N  3N', '2N 33N 4 5', '2N 33N 4', '2', '22N 33N', '3   4 5',
       '22N 3   4 5', '22N  3N 4 5', '3N 4 5', '2   33N 4 5', '3N', '4',
       '4 5', '2       4 5', '3N 4', '2   3   4 5', '22N     4', '3   4',
       '2N  3N 4', '2   3', '2N     4', '22N 33N   5', '33N   5', '5',
       '2N     4 5', '22N     4 5', '2    3N 4 5', '22N 3', '3',
       '2       4', '22N', '33N 4', '2   33N', '2N', '2N 3   4 5',
       '2N 3   4', '2   33N 4', '2N  3N   5', '33N', '2    3N 4',
       '22N 3     5', '3N   5', '22N  3N', '2         5', '2N 33N',
       '22N 3   4', '22N  3N 4', '2   3     5', '2   33N   5',
       '2N 3     5', '22N  3N   5', '1', '3     5', '2N 33N   5',
       '22N       5', '2    3N   5', '2N 3', '2    3N', '2N       5',
       None, '1 22N 33N 4 ', '22n 33n 4', '2 33N 4', '2 33N 4 5',
       '1 2 33N 4 5', '2  3N', '2  3N 4', '22n 33n 4 5', '2 33N',
       '2     4', '2 3', '2 3   4 5', '2 3

In [9]:
NPI_QUERY = \
    f"""
    SELECT
    PARTY_ID,
    NPI_CD,
    ENUMERATION_DT,
    DEACTIVATE_DT,
    REACTIVATE_DT
    FROM
    AMAEDW.NPI_REG
    WHERE
    THRU_DT is null
    """
npi = pd.read_sql(con=AMAEDW, sql=NPI_QUERY)

In [15]:
#LICENSE
today = datetime.date.today()
types = ['Physician','Student','Resident']
all_mes = list(ALL_LICENSE.ME)
elements = [
    {'data_element': 'SLN License State',
     'element':'STATE_ID',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'SLN License Status',
     'element':'STS_TYPE_ID',
    'null': ['',43,-1],
    'universe': all_mes},
    {'data_element': 'SLN License Type',
     'element':'LIC_TYPE_ID',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'SLN Issue Date',
     'element':'ISS_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'SLN Expiration Date',
     'element':'EXP_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'SLN Renewal Date',
     'element':'RNW_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'SLN License Degree',
     'element':'DEGREE_CD',
    'null': ['',-1],
    'universe': all_mes}
]
dict_list = []
for typo in types:
    total = len(ALL_LICENSE[ALL_LICENSE.type==typo])
    for element in elements:
        universe = len(ALL_LICENSE[(ALL_LICENSE.type==typo)&(ALL_LICENSE.ME.isin(element['universe']))])
        complete = len(ALL_LICENSE[(ALL_LICENSE.type==typo)&~(ALL_LICENSE[element['element']].isna())&(ALL_LICENSE.ME.isin(element['universe']))])
        true_complete = len(ALL_LICENSE[(ALL_LICENSE.type==typo)&~(ALL_LICENSE[element['element']].isin(element['null']))&~(ALL_LICENSE[element['element']].isna())&(ALL_LICENSE.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness',
            'Credential':'SLN'
        }
        dict_list.append(new_dict)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
#ABMS
all_mes = list(ALL_ABMS.CERTIF_ID)
elements = [
    {'data_element': 'ABMS Board Identifier',
     'element':'BRD_ID',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'ABMS Certificate Type',
     'element':'CERTIF_TYPE_CD',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'ABMS Duration Type',
     'element':'CERTIF_ID',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'ABMS Status',
     'element':'REC_TYPE',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'ABMS Certification Date',
     'element':'ISS_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'ABMS Expiration Date',
     'element':'EXP_DT',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'ABMS Reverification Date',
     'element':'REVERIFICATION_DT',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'ABMS Occurence',
     'element':'OCCUR_TYPE_CD',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'ABMS Participating in MOC',
     'element':'MOC_MET_RQT',
    'null': ['', 'N  '],
    'universe': all_mes},
    {'data_element': 'ABMS MOC Pathway ID',
     'element':'MOC_PATHWAY_ID',
    'null': ['',-1],
    'universe': all_mes}]
for typo in types:
    total = len(ALL_ABMS[ALL_ABMS.type==typo])
    for element in elements:
        universe = len(ALL_ABMS[(ALL_ABMS.type==typo)&(ALL_ABMS.CERTIF_ID.isin(element['universe']))])
        complete = len(ALL_ABMS[(ALL_ABMS.type==typo)&~(ALL_ABMS[element['element']].isna())&(ALL_ABMS.CERTIF_ID.isin(element['universe']))])
        true_complete = len(ALL_ABMS[(ALL_ABMS.type==typo)&~(ALL_ABMS[element['element']].isin(element['null']))&~(ALL_ABMS[element['element']].isna())&(ALL_ABMS.CERTIF_ID.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness',
            'Credential':'ABMS'
        }
        dict_list.append(new_dict)

In [16]:
#DEA
all_mes = list(ALL_DEA.ME)
elements = [
    {'data_element': 'DEA Business Activity Code',
     'element':'DEA_ACTIVITY_CD',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'DEA Business Activity Sub-Code',
     'element':'DEA_SUB_ACTIVITY_CODE',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'DEA Schedule Code',
     'element':'SCHED_CD',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'DEA Expiration Date',
     'element':'EXP_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'DEA Status',
     'element':'DEA_STATUS_ID',
    'null': ['',-1, 3],
    'universe': all_mes}]
for typo in types:
    total = len(ALL_DEA[ALL_DEA.type==typo])
    for element in elements:
        universe = len(ALL_DEA[(ALL_DEA.type==typo)&(ALL_DEA.ME.isin(element['universe']))])
        complete = len(ALL_DEA[(ALL_DEA.type==typo)&~(ALL_DEA[element['element']].isna())&(ALL_DEA.ME.isin(element['universe']))])
        true_complete = len(ALL_DEA[(ALL_DEA.type==typo)&~(ALL_DEA[element['element']].isin(element['null']))&~(ALL_DEA[element['element']].isna())&(ALL_DEA.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness',
            'Credential':'DEA'
        }
        dict_list.append(new_dict)

NameError: name 'ALL_DEA' is not defined

In [18]:
#NPI
all_mes = list(ALL_NPI.ME)
elements = [
    {'data_element': 'NPI Enumeration Date',
     'element':'ENUMERATION_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'NPI Deactivate Date',
     'element':'DEACTIVATE_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'NPI Reactivate DT',
     'element':'REACTIVATE_DT',
    'null': ['',-1],
    'universe': all_mes}]
for typo in types:
    total = len(ALL_NPI[ALL_NPI.type==typo])
    for element in elements:
        universe = len(ALL_NPI[(ALL_NPI.type==typo)&(ALL_NPI.ME.isin(element['universe']))])
        complete = len(ALL_NPI[(ALL_NPI.type==typo)&~(ALL_NPI[element['element']].isna())&(ALL_NPI.ME.isin(element['universe']))])
        true_complete = len(ALL_NPI[(ALL_NPI.type==typo)&~(ALL_NPI[element['element']].isin(element['null']))&~(ALL_NPI[element['element']].isna())&(ALL_NPI.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness',
            'Credential':'NPI'
        }
        dict_list.append(new_dict)

In [20]:
pd.DataFrame(dict_list).to_csv(f'../../Data/Measurement/Credential_Record_Completeness_{str(datetime.date.today())}.csv', index=False)

In [ ]:
import math
def coin_reward(minutes):
    x = 4+ 0.2*minutes + 5*(minutes-30)/30
    return math.floor(x)

In [108]:
folders_2 = pd.read_csv('../../Data/Measurement/License_Folder_Creation_July.csv')
dict_list = folders_2.to_dict('records')

In [111]:
for x in dict_list:
    thing = x['STATE'].split(' ')[-1]
    if thing in ['DO','MD']:
        license_degree = thing
        state = x['STATE'].replace(f' {thing}','')
    else:
        license_degree = ''
        state = x['STATE']
    if thing in ['TR','IR']:
        lic_type = 'Resident'
        state = x['STATE'].replace(f' {thing}','')
    else:
        lic_type = ''
    x['LICENSE_TYPE'] = lic_type
    x['LICENSE_DEGREE'] = license_degree
    x['LIC_STATE'] = state

In [112]:
dict_list

[{'STATE': 'Alabama',
  'CREATION_DT': '08:17.1',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': '',
  'LIC_STATE': 'Alabama'},
 {'STATE': 'Alaska',
  'CREATION_DT': '21:24.0',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': '',
  'LIC_STATE': 'Alaska'},
 {'STATE': 'Arizona DO',
  'CREATION_DT': '40:51.6',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': 'DO',
  'LIC_STATE': 'Arizona'},
 {'STATE': 'Arizonia MD',
  'CREATION_DT': '35:16.6',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': 'MD',
  'LIC_STATE': 'Arizonia'},
 {'STATE': 'Arkansas',
  'CREATION_DT': '33:22.4',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': '',
  'LIC_STATE': 'Arkansas'},
 {'STATE': 'California MD',
  'CREATION_DT': '43:14.0',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': 'MD',
  'LIC_STATE': 'California'},
 {'STATE': 'Colorado',
  'CREATION_DT': '54:55.7',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': '',
  'LIC_STATE': 'Colorado'},
 {'STATE': 'Delaware',
  'CREATION_DT': '48:15.3',
  'LICENSE_TYPE': '',
  'LICENSE_DEGREE': '',
  'LIC_STATE': 'D